<img src="../files/misc/logo.png" width=300/>
<h1 style="color:#872325"> Natural Language Processing </h1>

In [1]:
import nltk
import pandas as pd
import numpy as np
from collections import Counter

**Natural Language**
> _a language that has developed naturally in use (as contrasted with an artificial language or computer code)._

---

Existen dos principales razones por las cuáles estamos interesados en hacer programas de computación que entiendan lenguaje natural:
1. Comunicación entre humanos
2. Para obtener información de un lenguaje escrito

En el caso de desear obtener información, podemos dividir esta clase de problemas en tres sub-problemas:

- Clasificación de Textos
- Aquisición de información
- Extracción de información

Para resolver estas clases de problemas haremos uso de los **modelos de lenguaje** los cuales estiman distribuciones de probabilidad sobre expresiones del lenguaje

* Definimos un modelo de lenguaje natural como una distribución de probabilidad sobre oraciones

## Palabras, _corpus_ y _corpora_

**corpus**: a computer-readable collection of text or speech.  
**corpora**: plural de corpus

### Completa la frase
Ayer vi a mi...

# El modelo $n$-gram

La finalidad de un modelo $n$-gram es modelar una secuencia de $n$ elementos (palabras, en este caso) dada una muestra de texto. Para lograr esto, el modelo asume que la probabilidad del acontecimiento de una palabra, dada toda una historia de palabras que la preceden, es igual a la probabilidad de esa misma palabra dada las últimas $n$ palabras que se dijeron.

Formalmente, un modelo $n$-gram se define como una Cadena de Markov de order $n-1$. Esto es,

$$
    \mathbb P(W_m \ | \  W_{m-1}, W_{m-2}, \ldots, W_{1}) = \mathbb P(W_m \ | \ W_{m-1}, W_{m-2}, \ldots, W_{m-n+1})
$$


Dado un texto (o corpus) con $m$ palabras o carácteres y $n \leq m$,


Dado esto, vemos que un $1$-gram (o *unigram* asume que)
$$
\mathbb P(W_m \ | \  W_{m-1}, W_{m-2}, \ldots, W_{1}) = \mathbb P(X_m)
$$

un $2$-gram (o bigram), el cuál cumple la [propiedad de Markov](https://es.wikipedia.org/wiki/Cadena_de_Márkov), asume que,
$$
\mathbb P(W_m \ | \  W_{m-1}, W_{m-2}, \ldots, W_{1}) = \mathbb P(X_m \ | \ X_{m-1})
$$

un $3$-gram,
$$
    \mathbb P(W_m \ | \  W_{m-1}, W_{m-2}, \ldots, W_{1}) = \mathbb P(X_m \ | \ X_{m-1}, X_{m-2})
$$

y así sucesivamente.

La construcción de elementos para un modelo $n$-gram puede ser hehca considerando carácter por carácter o palabra por palabra. A cada elemento le llamamos un **token**.

In [2]:
# Creación de 3-grams carácter por carácter
sentence = "War is peace freedom is slavery ignorance is strength."
for gram in nltk.ngrams(sentence, 3):
    print(gram)

('W', 'a', 'r')
('a', 'r', ' ')
('r', ' ', 'i')
(' ', 'i', 's')
('i', 's', ' ')
('s', ' ', 'p')
(' ', 'p', 'e')
('p', 'e', 'a')
('e', 'a', 'c')
('a', 'c', 'e')
('c', 'e', ' ')
('e', ' ', 'f')
(' ', 'f', 'r')
('f', 'r', 'e')
('r', 'e', 'e')
('e', 'e', 'd')
('e', 'd', 'o')
('d', 'o', 'm')
('o', 'm', ' ')
('m', ' ', 'i')
(' ', 'i', 's')
('i', 's', ' ')
('s', ' ', 's')
(' ', 's', 'l')
('s', 'l', 'a')
('l', 'a', 'v')
('a', 'v', 'e')
('v', 'e', 'r')
('e', 'r', 'y')
('r', 'y', ' ')
('y', ' ', 'i')
(' ', 'i', 'g')
('i', 'g', 'n')
('g', 'n', 'o')
('n', 'o', 'r')
('o', 'r', 'a')
('r', 'a', 'n')
('a', 'n', 'c')
('n', 'c', 'e')
('c', 'e', ' ')
('e', ' ', 'i')
(' ', 'i', 's')
('i', 's', ' ')
('s', ' ', 's')
(' ', 's', 't')
('s', 't', 'r')
('t', 'r', 'e')
('r', 'e', 'n')
('e', 'n', 'g')
('n', 'g', 't')
('g', 't', 'h')
('t', 'h', '.')


In [3]:
# Creación de 3-grams palabra por palabra
sentence = "who controls the past controls the future: who controls the present controls the past."
for gram in nltk.ngrams(sentence.split(), 3):
    print(gram)

('who', 'controls', 'the')
('controls', 'the', 'past')
('the', 'past', 'controls')
('past', 'controls', 'the')
('controls', 'the', 'future:')
('the', 'future:', 'who')
('future:', 'who', 'controls')
('who', 'controls', 'the')
('controls', 'the', 'present')
('the', 'present', 'controls')
('present', 'controls', 'the')
('controls', 'the', 'past.')


### Estimando Empíricamente las Probabilidades de un Texto

Dado un modelo $n$-gram, estimamos la probabilidad de un *token* $w_n$ de la siguiente manera

$$
\begin{align}
    p(w_k | w_{k-1}, \ldots, w_{k-n+1}) &= \frac{C(w_k, w_{k-1}, \ldots, w_{k-n+1})}{\sum_{w} C(w, w_{k-1}, \ldots, w_{k-n+1})}\\
    &= \frac{C(w_k, w_{k-1}, \ldots, w_{k-n+1})}{C(w_{k-1}, \ldots, w_{k-n+1})}
\end{align}
$$

Donde
* $C(\cdot)$ es la frequencia del $n$-gram

In [5]:
from nltk.tokenize import RegexpTokenizer

work_tokenizer = RegexpTokenizer("\w+")
tokens = work_tokenizer.tokenize(sentence)
tokens

['who',
 'controls',
 'the',
 'past',
 'controls',
 'the',
 'future',
 'who',
 'controls',
 'the',
 'present',
 'controls',
 'the',
 'past']

In [6]:
gram2 = list(nltk.ngrams(tokens, 2))
gram2

[('who', 'controls'),
 ('controls', 'the'),
 ('the', 'past'),
 ('past', 'controls'),
 ('controls', 'the'),
 ('the', 'future'),
 ('future', 'who'),
 ('who', 'controls'),
 ('controls', 'the'),
 ('the', 'present'),
 ('present', 'controls'),
 ('controls', 'the'),
 ('the', 'past')]

In [9]:
unique_tokens = list(set(tokens))
unique_tokens

['future', 'controls', 'who', 'the', 'present', 'past']

### Construyendo una matriz de frequencias

In [10]:
counts = {tok: Counter([g[:-1] for g in gram2 if g[-1] in tok]) for tok in unique_tokens}
counts = pd.DataFrame(counts).fillna(0)
counts

,future,controls,who,the,present,past
controls,0.0,0.0,0.0,4.0,0.0,0.0
future,0.0,0.0,1.0,0.0,0.0,0.0
past,0.0,1.0,0.0,0.0,0.0,0.0
present,0.0,1.0,0.0,0.0,0.0,0.0
the,1.0,0.0,0.0,0.0,1.0,2.0
who,0.0,2.0,0.0,0.0,0.0,0.0


In [11]:
# Probability mass table
pm_table = counts / counts.sum(axis=1).values[:, np.newaxis]
pm_table

,future,controls,who,the,present,past
controls,0.00,0.0,0.0,1.0,0.00,0.0
future,0.00,0.0,1.0,0.0,0.00,0.0
past,0.00,1.0,0.0,0.0,0.00,0.0
present,0.00,1.0,0.0,0.0,0.00,0.0
the,0.25,0.0,0.0,0.0,0.25,0.5
who,0.00,1.0,0.0,0.0,0.00,0.0


In [13]:
# P('past' | 'the ')
pm_table.loc["the", "past"]

the    0.5
Name: past, dtype: float64

In [14]:
# P(w | 'the ')
pm_table.loc["the", :]

,future,controls,who,the,present,past
the,0.25,0.0,0.0,0.0,0.25,0.5


In [18]:
# Σw P(w | 'the ') = 1
pm_table.loc["the", :].sum()

future      0.25
controls    0.00
who         0.00
the         0.00
present     0.25
past        0.50
dtype: float64

In [19]:
# P('controls' | 'who')
pm_table.loc["who", "controls"]

who    1.0
Name: controls, dtype: float64

In [20]:
# P(who U past | controls)
pm_table.loc["who", ["controls", "future"]].sum(axis=1)

who    1.0
dtype: float64

<h2 style="color:teal">Ejemplo: 1984</h2>

In [21]:
import requests

In [22]:
url = "http://www.gutenberg.org/cache/epub/2000/pg2000.txt"
url = "http://gutenberg.net.au/ebooks01/0100021.txt"
r = requests.get(url)
corpus = r.text

**Acotamos el texto**

In [23]:
init_book = corpus.find("Chapter 1")
end_book = corpus.find("End of Project Gutenberg's")
corpus = corpus[init_book: end_book]

In [24]:
print(corpus[:500])

Chapter 1



It was a bright cold day in April, and the clocks were striking thirteen.
Winston Smith, his chin nuzzled into his breast in an effort to escape the
vile wind, slipped quickly through the glass doors of Victory Mansions,
though not quickly enough to prevent a swirl of gritty dust from entering
along with him.

The hallway smelt of boiled cabbage and old rag mats. At one end of it a
coloured poster, too large for indoor display, had been tacked to the wall.
It depicted si


In [25]:
tokens = work_tokenizer.tokenize(corpus.lower())
tokens[:10]

['chapter', '1', 'it', 'was', 'a', 'bright', 'cold', 'day', 'in', 'april']

In [26]:
unique_tokens = list(set(tokens))
len(unique_tokens)

8864

## Estimando Frequencias

In [27]:
def value_counts(token, gram):
    gram_counter = Counter([g[:-1] for g in gram if g[-1] == token])
    return (token, gram_counter)

In [28]:
%%time
gram3 = list(nltk.ngrams(tokens, 3))
counts = {tok: value_counts(tok, gram3) for tok in unique_tokens}

CPU times: user 41.8 s, sys: 77.8 ms, total: 41.8 s
Wall time: 42 s


In [31]:
%%time
from multiprocessing import Pool


gram3 = list(nltk.ngrams(tokens, 3))

def value_counts(token):
    gram_counter = Counter([g[:-1] for g in gram3 if g[-1] == token])
    return (token, gram_counter)

pool = Pool(processes=6)
res = pool.map(value_counts, unique_tokens)
counter = dict(res)
pool.close()

CPU times: user 245 ms, sys: 93 ms, total: 338 ms
Wall time: 23.9 s


In [45]:
import os
os.cpu_count()

4

In [53]:
%%time
# Count DataFrame
cdf = pd.DataFrame(counter)

CPU times: user 38.7 s, sys: 5.68 s, total: 44.4 s
Wall time: 46.5 s


Calculando la probabilidad de 

$$
    p(\texttt{you}, \texttt{are}, \texttt{the})
$$

In [54]:
# p('you')
p_a = cdf["you"].sum() / np.nansum(cdf.values)
p_a

0.00958057729848568

In [55]:
# p('are' | 'you')
p_b = cdf.xs("you", level=1)["are"].sum() / cdf["you"].sum()
p_b

0.06429277942631058

In [56]:
# p('the' | 'you', 'are')
p_c = cdf.loc[("you", "are"), "the"] / cdf.loc[("you", "are")].sum()
p_c

0.07692307692307693

In [57]:
p = p_a * p_b * p_c
format(p, "0.8%")

'0.00473817%'

In [61]:
r = [v for v in gram3 if v == ("you", "are", "the")]
print(r)
p = len(r) / len(gram3)
format(p, "0.8%")

[('you', 'are', 'the'), ('you', 'are', 'the'), ('you', 'are', 'the'), ('you', 'are', 'the'), ('you', 'are', 'the')]


'0.00473817%'

## Generando un Nuevo Texto

In [62]:
from numpy.random import choice, seed

In [63]:
def sample_gram(tokens, gram):
    tokens = tuple(tokens)
    counts = cdf.loc[tokens].dropna()
    probs = counts / counts.sum()
    return probs.index , probs.values

In [64]:
sentence = "i am"

for i in range(15):
    toks = work_tokenizer.tokenize(sentence)
    words, ps = sample_gram(toks[-2:], gram=3)
    
    seed(1643 + i)
    w = choice(words, p=ps)
    toks.append(w)
    toks.pop(0)

    sentence += f" {w}"

In [65]:
sentence

'i am well aware you are finally caught you it will be made clear later in the'

# Naïve Bayes

## Un Clasificador Probabilistico

Sea $d=(f_1, \ldots, f_n)$ un vector de características de un documento $d$, $C = \{c_k\}_{k=1}^K$ una colección de clases para un documento.

> Dado un documento $d$, queremos estimar la clase a la cuál pertenece $c$

El modelo de Naïve Bayes hace dos supociones sobre un documento $d$:
1. *Bag of Words assumption*: El orden dentro del documento no importa
2. *Naïve Bayes assumption*: Cada característica del modelo es independiente

Considerando las dos supociones del modelo, estimamos la clase $c_k$ de un documento de la siguiente manera:

$$
\begin{align}
    \hat c &= \arg\max_k p(C_k|d) \\
           &= \arg\max_k p(C_k) \prod_{n=1}^N p(W_n|C_k)
\end{align}
$$

## Referencias

- Jurafsky, Dan, and James H. Martin. *Speech and Language Processing: an Introduction to Natural Language Processing*, Computational Linguistics, and Speech Recognition. Dorling Kindersley Pvt, Ltd., 2014.

- Russell, Stuart J., and Peter Norvig. _Artificial Intelligence: a Modern Approach_. Pearson, 2021.